# Gradient-Boosted Tree Classification

## Imports

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler,StandardScaler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Start Spark

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/03/21 18:27:56 WARN Utils: Your hostname, MacBook-Pro-de-Matheo.local resolves to a loopback address: 127.0.0.1; using 10.10.19.136 instead (on interface en0)
23/03/21 18:27:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/21 18:27:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Load of train set and labels and merge

In [3]:
input_train_df = spark.read.csv("./InputTrain.csv",header=True,inferSchema=True)
labels_df = spark.read.csv("./StepOne_LabelTrain.csv",header=True,inferSchema=True)

merged_df = input_train_df.join(labels_df, on="Index",how="inner")

## Classification

### Creation of the assembler and apply on the merged data

In [8]:
# Define the feature columns and the target column
feature_cols = input_train_df.columns[2:]  # the first two columns are id and house id
target_col = "Washing Machine"
label_cols = ["Washing Machine", "Dishwasher", "Tumble Dryer", "Microwave", "Kettle"]

# Assemble the features into a vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(merged_df).select(*label_cols,col("features"))

### Extract randomly taken training set and a validation set from the data

In [9]:
# Split the data into training and validation sets
(training_data, validation_data) = data.randomSplit([0.8, 0.2], seed=42)

### Creation of the GBT and the model

In [10]:
# Train the GBT model
gbt = GBTClassifier(labelCol=target_col, featuresCol="features", maxIter=10)
model = gbt.fit(training_data)

23/03/21 18:38:24 WARN DAGScheduler: Broadcasting large task binary with size 1286.4 KiB
23/03/21 18:38:25 WARN DAGScheduler: Broadcasting large task binary with size 1287.1 KiB
23/03/21 18:38:25 WARN DAGScheduler: Broadcasting large task binary with size 1287.7 KiB
23/03/21 18:38:26 WARN DAGScheduler: Broadcasting large task binary with size 1288.8 KiB
23/03/21 18:38:26 WARN DAGScheduler: Broadcasting large task binary with size 1291.1 KiB
23/03/21 18:38:26 WARN DAGScheduler: Broadcasting large task binary with size 1297.6 KiB
23/03/21 18:38:26 WARN DAGScheduler: Broadcasting large task binary with size 1298.1 KiB
23/03/21 18:38:26 WARN DAGScheduler: Broadcasting large task binary with size 1298.7 KiB
23/03/21 18:38:27 WARN DAGScheduler: Broadcasting large task binary with size 1299.8 KiB
23/03/21 18:38:27 WARN DAGScheduler: Broadcasting large task binary with size 1302.1 KiB
23/03/21 18:38:27 WARN DAGScheduler: Broadcasting large task binary with size 1304.9 KiB
23/03/21 18:38:27 WAR

### Creation of the evaluator and compute of the predictions

In [11]:
# Evaluate the model on the validation set
evaluator = MulticlassClassificationEvaluator(labelCol=target_col, predictionCol="prediction", metricName="accuracy")
predictions = model.transform(validation_data)
accuracy = evaluator.evaluate(predictions)
print("Accuracy on validation set = %g" % accuracy)

Accuracy on validation set = 0.809735


### Test on `InputTest.csv`

In [24]:
test_data = spark.read.csv("InputTest.csv", header=True, inferSchema=True)
# Use the model to predict the state of the 5 appliances on the test data
test_features = assembler.transform(test_data).select(col("features"), col("Index"))
test_features.printSchema()

test_predictions = model.transform(test_features).select(col("prediction"), col("Index"))

# Join the predicted labels with the original input data using the index column
test_predictions = test_predictions.join(test_data, "Index")

test_predictions.printSchema()

# Select the desired columns
test_predictions = test_predictions.select(col("prediction"), *label_cols)

root
 |-- features: vector (nullable = true)
 |-- Index: integer (nullable = true)

root
 |-- Index: integer (nullable = true)
 |-- prediction: double (nullable = false)
 |-- House_id: integer (nullable = true)
 |-- TimeStep_0: double (nullable = true)
 |-- TimeStep_1: double (nullable = true)
 |-- TimeStep_2: double (nullable = true)
 |-- TimeStep_3: double (nullable = true)
 |-- TimeStep_4: double (nullable = true)
 |-- TimeStep_5: double (nullable = true)
 |-- TimeStep_6: double (nullable = true)
 |-- TimeStep_7: double (nullable = true)
 |-- TimeStep_8: double (nullable = true)
 |-- TimeStep_9: double (nullable = true)
 |-- TimeStep_10: double (nullable = true)
 |-- TimeStep_11: double (nullable = true)
 |-- TimeStep_12: double (nullable = true)
 |-- TimeStep_13: double (nullable = true)
 |-- TimeStep_14: double (nullable = true)
 |-- TimeStep_15: double (nullable = true)
 |-- TimeStep_16: double (nullable = true)
 |-- TimeStep_17: double (nullable = true)
 |-- TimeStep_18: double 

AnalysisException: cannot resolve '`Washing Machine`' given input columns: [House_id, Index, TimeStep_0, TimeStep_1, TimeStep_10, TimeStep_100, TimeStep_1000, TimeStep_1001, TimeStep_1002, TimeStep_1003, TimeStep_1004, TimeStep_1005, TimeStep_1006, TimeStep_1007, TimeStep_1008, TimeStep_1009, TimeStep_101, TimeStep_1010, TimeStep_1011, TimeStep_1012, TimeStep_1013, TimeStep_1014, TimeStep_1015, TimeStep_1016, TimeStep_1017, TimeStep_1018, TimeStep_1019, TimeStep_102, TimeStep_1020, TimeStep_1021, TimeStep_1022, TimeStep_1023, TimeStep_1024, TimeStep_1025, TimeStep_1026, TimeStep_1027, TimeStep_1028, TimeStep_1029, TimeStep_103, TimeStep_1030, TimeStep_1031, TimeStep_1032, TimeStep_1033, TimeStep_1034, TimeStep_1035, TimeStep_1036, TimeStep_1037, TimeStep_1038, TimeStep_1039, TimeStep_104, TimeStep_1040, TimeStep_1041, TimeStep_1042, TimeStep_1043, TimeStep_1044, TimeStep_1045, TimeStep_1046, TimeStep_1047, TimeStep_1048, TimeStep_1049, TimeStep_105, TimeStep_1050, TimeStep_1051, TimeSt

### Save of the predictions

In [7]:
# Save the predictions in the format specified in the instructions
test_predictions.write.csv("StepOne_Predictions.csv", header=True)

23/03/21 18:32:30 WARN DAGScheduler: Broadcasting large task binary with size 1286.4 KiB
23/03/21 18:32:32 WARN DAGScheduler: Broadcasting large task binary with size 1287.1 KiB
23/03/21 18:32:32 WARN DAGScheduler: Broadcasting large task binary with size 1287.7 KiB
23/03/21 18:32:32 WARN DAGScheduler: Broadcasting large task binary with size 1288.8 KiB
23/03/21 18:32:32 WARN DAGScheduler: Broadcasting large task binary with size 1291.1 KiB
23/03/21 18:32:32 WARN DAGScheduler: Broadcasting large task binary with size 1297.6 KiB
23/03/21 18:32:32 WARN DAGScheduler: Broadcasting large task binary with size 1298.1 KiB
23/03/21 18:32:33 WARN DAGScheduler: Broadcasting large task binary with size 1298.7 KiB
23/03/21 18:32:33 WARN DAGScheduler: Broadcasting large task binary with size 1299.8 KiB
23/03/21 18:32:33 WARN DAGScheduler: Broadcasting large task binary with size 1302.1 KiB
23/03/21 18:32:33 WARN DAGScheduler: Broadcasting large task binary with size 1304.9 KiB
23/03/21 18:32:33 WAR

Accuracy on validation set = 0.809735
root
 |-- features: vector (nullable = true)



AnalysisException: cannot resolve '`predictions`' given input columns: [features, prediction, probability, rawPrediction];;
'Project ['predictions, 'Washing Machine, 'Dishwasher, 'Tumble Dryer, 'Microwave, 'Kettle]
+- Project [features#65370, rawPrediction#67536, probability#67541, UDF(rawPrediction#67536) AS prediction#67547]
   +- Project [features#65370, rawPrediction#67536, UDF(rawPrediction#67536) AS probability#67541]
      +- Project [features#65370, UDF(features#65370) AS rawPrediction#67536]
         +- Project [features#65370]
            +- Project [Index#61046, House_id#61047, TimeStep_0#61048, TimeStep_1#61049, TimeStep_2#61050, TimeStep_3#61051, TimeStep_4#61052, TimeStep_5#61053, TimeStep_6#61054, TimeStep_7#61055, TimeStep_8#61056, TimeStep_9#61057, TimeStep_10#61058, TimeStep_11#61059, TimeStep_12#61060, TimeStep_13#61061, TimeStep_14#61062, TimeStep_15#61063, TimeStep_16#61064, TimeStep_17#61065, TimeStep_18#61066, TimeStep_19#61067, TimeStep_20#61068, TimeStep_21#610